In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('../001/train.pk1')

In [3]:
train_pkl.shape

(850, 11)

In [4]:
# 年齢、性別は削除
train_pkl.drop(['Age', 'Gender_Male'], axis=1, inplace=True)

In [5]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0


In [6]:
train_pk2 = train_pkl.copy()
train_pkl.shape

(850, 9)

In [7]:
train_pkl = train_pk2.copy()
loop = train_pkl.shape[1] - 1
for i in range(loop):
    for j in range(i+1, loop):
        colname = train_pkl.columns[i] + "_" + train_pkl.columns[j] 
        train_pkl["div" + "_" + colname] = train_pkl.iloc[:, i] / train_pkl.iloc[:, j]

In [8]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,div_T_Bil_D_Bil,...,div_ALT_GPT_AST_GOT,div_ALT_GPT_TP,div_ALT_GPT_Alb,div_ALT_GPT_AG_ratio,div_AST_GOT_TP,div_AST_GOT_Alb,div_AST_GOT_AG_ratio,div_TP_Alb,div_TP_AG_ratio,div_Alb_AG_ratio
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0,5.230469,...,0.619792,1.975765,4.328575,13.376179,3.187786,6.983913,21.581715,2.191406,6.769531,3.089844
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1,5.128906,...,0.788133,7.406217,16.727999,67.911980,9.397162,21.224833,86.168129,2.257812,9.171875,4.058594
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0,8.085938,...,0.412404,2.122129,5.073788,16.292391,5.145748,12.302942,39.505856,2.390625,7.675781,3.210938
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1,4.234375,...,0.630115,4.928803,9.505930,34.754883,7.822075,15.086035,55.156452,1.928711,7.050781,3.656250
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0,8.773438,...,0.121010,3.525208,6.704851,20.044378,29.131598,55.407516,165.642654,1.902344,5.687500,2.990234


In [10]:
train_pkl.describe()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,div_T_Bil_D_Bil,...,div_ALT_GPT_AST_GOT,div_ALT_GPT_TP,div_ALT_GPT_Alb,div_ALT_GPT_AG_ratio,div_AST_GOT_TP,div_AST_GOT_Alb,div_AST_GOT_AG_ratio,div_TP_Alb,div_TP_AG_ratio,div_Alb_AG_ratio
count,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,...,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000
mean,1.607422,0.606445,272.211334,41.978474,61.018120,7.031250,3.517578,1.144531,0.444706,4.855469,...,0.676123,6.316390,12.781148,37.539436,8.938665,18.218481,56.111256,2.023438,6.351562,3.138672
std,2.443359,1.568359,200.348511,145.927368,110.125389,0.840820,0.560059,0.227295,0.497226,2.738281,...,0.818627,23.702866,47.621643,127.521683,16.541906,33.915825,99.008430,0.230347,1.327148,0.528320
min,0.585938,0.034851,163.261841,3.924518,11.283497,4.859375,2.181641,0.626953,0.000000,0.071350,...,0.041492,0.558775,1.125688,3.030699,1.633809,3.158639,9.626768,1.469727,3.679688,1.948242
25%,0.781372,0.144928,213.991409,13.644659,21.239157,6.730469,3.138672,1.004883,0.000000,3.299805,...,0.381331,1.935682,3.905817,11.455967,2.868896,5.738187,16.844880,1.866211,5.479492,2.807129
50%,0.835693,0.194092,220.092506,16.638000,27.056026,6.914062,3.621094,1.205078,0.000000,4.318359,...,0.598122,2.417187,4.821839,15.017028,4.077625,8.397087,25.452895,2.041016,5.888672,3.078125
75%,1.197021,0.335388,229.455933,23.056081,56.461569,7.535156,3.712891,1.288086,1.000000,5.862305,...,0.749566,3.420392,7.028483,22.351111,8.262585,16.605571,51.227290,2.140625,7.045898,3.495117
max,23.015625,17.687500,2108.483643,1423.186523,814.439392,8.742188,5.015625,1.821289,1.000000,44.593750,...,13.121293,235.359009,515.859924,1436.501587,120.170883,259.170898,854.381470,3.443359,10.875000,5.175781


In [11]:
disease_bk = train_pkl.disease.values

In [12]:
train_pkl.drop('disease', axis=1, inplace=True)

In [13]:
train_pkl['disease'] = disease_bk

In [14]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,div_T_Bil_D_Bil,div_T_Bil_ALP,...,div_ALT_GPT_TP,div_ALT_GPT_Alb,div_ALT_GPT_AG_ratio,div_AST_GOT_TP,div_AST_GOT_Alb,div_AST_GOT_AG_ratio,div_TP_Alb,div_TP_AG_ratio,div_Alb_AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,5.230469,0.003575,...,1.975765,4.328575,13.376179,3.187786,6.983913,21.581715,2.191406,6.769531,3.089844,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,5.128906,0.004538,...,7.406217,16.727999,67.911980,9.397162,21.224833,86.168129,2.257812,9.171875,4.058594,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,8.085938,0.002049,...,2.122129,5.073788,16.292391,5.145748,12.302942,39.505856,2.390625,7.675781,3.210938,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,4.234375,0.002455,...,4.928803,9.505930,34.754883,7.822075,15.086035,55.156452,1.928711,7.050781,3.656250,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,8.773438,0.007789,...,3.525208,6.704851,20.044378,29.131598,55.407516,165.642654,1.902344,5.687500,2.990234,0


## 訓練データとテストデータに分割

In [15]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [16]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1]
train_y = train.disease.values

In [17]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

## 訓練データで訓練

In [42]:
XGB = XGBClassifier(lambda_l1=300, lamba_l2=5, max_depth=7)

In [43]:
XGB = XGB.fit(train_X, train_y, early_stopping_rounds=20, eval_metric='auc', eval_set=[(test_x, test_y)])

[0]	validation_0-auc:0.953847
Will train until validation_0-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.953987
[2]	validation_0-auc:0.95294
[3]	validation_0-auc:0.956431
[4]	validation_0-auc:0.958944
[5]	validation_0-auc:0.955663
[6]	validation_0-auc:0.954895
[7]	validation_0-auc:0.952381
[8]	validation_0-auc:0.953638
[9]	validation_0-auc:0.954057
[10]	validation_0-auc:0.954196
[11]	validation_0-auc:0.95671
[12]	validation_0-auc:0.956431
[13]	validation_0-auc:0.958386
[14]	validation_0-auc:0.957687
[15]	validation_0-auc:0.957687
[16]	validation_0-auc:0.957269
[17]	validation_0-auc:0.958246
[18]	validation_0-auc:0.95671
[19]	validation_0-auc:0.956151
[20]	validation_0-auc:0.957967
[21]	validation_0-auc:0.958944
[22]	validation_0-auc:0.959084
[23]	validation_0-auc:0.958525
[24]	validation_0-auc:0.958805
[25]	validation_0-auc:0.959363
[26]	validation_0-auc:0.959643
[27]	validation_0-auc:0.959782
[28]	validation_0-auc:0.96048
[29]	validation_0-auc:0.961598
[30]	validation_0-au

In [44]:
XGB.feature_importances_

array([0.04432471, 0.01573031, 0.01454753, 0.02328334, 0.02171885,
       0.01439727, 0.00676722, 0.01270699, 0.01096767, 0.01724879,
       0.00915131, 0.01938874, 0.01715811, 0.02156118, 0.01044676,
       0.01263664, 0.02129953, 0.01728514, 0.01600542, 0.00931416,
       0.16442715, 0.01590722, 0.01976719, 0.01791727, 0.01076724,
       0.01291386, 0.01074833, 0.04038562, 0.03763998, 0.25501898,
       0.01328172, 0.01347879, 0.01778252, 0.00861201, 0.01579454,
       0.00961792], dtype=float32)

In [45]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train_X.iloc[:, :].columns),
    reverse=True)

[(0.255, 'div_ALT_GPT_AG_ratio'),
 (0.164, 'div_D_Bil_AG_ratio'),
 (0.044, 'T_Bil'),
 (0.04, 'div_ALT_GPT_TP'),
 (0.038, 'div_ALT_GPT_Alb'),
 (0.023, 'ALT_GPT'),
 (0.022, 'div_T_Bil_Alb'),
 (0.022, 'AST_GOT'),
 (0.021, 'div_D_Bil_ALT_GPT'),
 (0.02, 'div_ALP_AST_GOT'),
 (0.019, 'div_T_Bil_AST_GOT'),
 (0.018, 'div_AST_GOT_AG_ratio'),
 (0.018, 'div_ALP_TP'),
 (0.017, 'div_T_Bil_TP'),
 (0.017, 'div_T_Bil_ALP'),
 (0.017, 'div_D_Bil_AST_GOT'),
 (0.016, 'div_TP_AG_ratio'),
 (0.016, 'div_D_Bil_TP'),
 (0.016, 'div_ALP_ALT_GPT'),
 (0.016, 'D_Bil'),
 (0.015, 'ALP'),
 (0.014, 'TP'),
 (0.013, 'div_D_Bil_ALP'),
 (0.013, 'div_AST_GOT_TP'),
 (0.013, 'div_AST_GOT_Alb'),
 (0.013, 'div_ALP_AG_ratio'),
 (0.013, 'AG_ratio'),
 (0.011, 'div_T_Bil_D_Bil'),
 (0.011, 'div_ALT_GPT_AST_GOT'),
 (0.011, 'div_ALP_Alb'),
 (0.01, 'div_T_Bil_AG_ratio'),
 (0.01, 'div_Alb_AG_ratio'),
 (0.009, 'div_T_Bil_ALT_GPT'),
 (0.009, 'div_TP_Alb'),
 (0.009, 'div_D_Bil_Alb'),
 (0.007, 'Alb')]

## テストデータで実行

In [46]:
test_x.shape, test_y.shape

((170, 36), (170,))

In [47]:
pred_y = XGB.predict(test_x)

In [48]:
confusion_matrix(test_y, pred_y)

array([[88,  5],
       [13, 64]], dtype=int64)

In [49]:
accuracy_score(test_y, pred_y)

0.8941176470588236

## 検証データで実行

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('../001/test.pk1')

In [ ]:
valid.shape

In [ ]:
type(valid)

In [ ]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [ ]:
valid_X.head()

In [ ]:
# 年齢、性別は削除
valid_X.drop(['Age', 'Gender_Male'], axis=1, inplace=True)

In [ ]:
valid_X.shape

In [ ]:
loop = valid_X.shape[1]
for i in range(loop):
    for j in range(i+1, loop):
        colname = valid_X.columns[i] + "_" + valid_X.columns[j] 
        valid_X["mul" + "_" + colname] = valid_X.iloc[:, i] * valid_X.iloc[:, j]

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
valid_X.head()

In [ ]:
pred_valid_y = XGB.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
# type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y)

In [ ]:
result_df.to_csv("./XGB_6.csv", header=False)

In [ ]:
pred_valid_y.sum()